# Task 2 : Klasifikasi Topik Artikel Tanpa coloumn article_topic

## Load Library

In [1]:
import csv,numpy,pandas,nltk
import sklearn
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk import sent_tokenize, word_tokenize, pos_tag

from sklearn.cross_validation import StratifiedKFold, cross_val_score
from sklearn.decomposition import LatentDirichletAllocation,NMF
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

c:\users\user\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Load Data

In [2]:
#Baca data CSV dengan pandas
df = pandas.read_csv('../data/ds_asg_data.csv')

df.head()

,article_id,article_topic,article_content
0,93205794,Internasional,Kepolisian Inggris tengah memburu pelaku yang...
1,93186698,Ekonomi,Seluruh layanan transaksi di jalan tol akan m...
2,93191463,Teknologi,"\nHari ini, Rabu (23/8), ternyata menjadi har..."
3,93219292,Ekonomi,Saat ini Indonesia hanya memiliki cadangan ba...
4,343106,Hiburan,"Hari ini, Selasa (1/8), pedangdut Ridho Rhoma..."


In [3]:
#View berdasar topic
groupby= df.groupby('article_topic').size()
print (groupby)

article_topic
Bisnis             25
Bojonegoro        260
Ekonomi          1762
Haji             1497
Health            131
Hiburan          1466
Horor              50
Hukum              85
Internasional     741
Jakarta            12
K-Pop              61
KPK                37
Kesehatan         195
Keuangan           14
Lifestyle         572
MotoGP             35
Obat-obatan        58
Otomotif          174
Pendidikan         70
Personal           81
Pilgub Jatim       25
Politik           104
Regional           35
Sains             174
Sejarah            70
Sepak Bola       1184
Sports            435
Teknologi         571
Travel             76
dtype: int64


View berdasarkan topik dan didapatkan 29 topik dengan topik Ekonomi mempunyai proporsi paling banyak, dan dapat dilihat data tersebut tidak proporsional

Dicek dahulu apakah data terdapat missing value

In [4]:
#Cek data dan normalisasi
#cek bila ada yang kosong
cek_bentuk= df['article_content'].shape
cek_null=df.isnull()
cek_jumlah_null=df.isnull().sum()

print (cek_bentuk)
print (cek_null)
print (cek_jumlah_null)

(10000,)
      article_id  article_topic  article_content
0          False          False            False
1          False          False            False
2          False          False            False
3          False          False            False
4          False          False            False
5          False          False            False
6          False          False            False
7          False          False            False
8          False          False            False
9          False          False            False
10         False          False            False
11         False          False            False
12         False          False            False
13         False          False            False
14         False          False            False
15         False          False            False
16         False          False            False
17         False          False            False
18         False          False            False
19         

Setelah cek data apakah ada data yang null atau tidak, karena ada data yang null/missing value maka perlu dinormalisasi terlebih dahulu.

Kita hapus data yang terdapat missing value

In [5]:
#Drop/Hapus data yang kosong
modifiedData = df.dropna()

cek_bentuk= modifiedData['article_content'].shape
cek_null=modifiedData.isnull()
cek_jumlah_null=modifiedData.isnull().sum()

print (cek_bentuk)
print (cek_null)
print (cek_jumlah_null)

(9964,)
      article_id  article_topic  article_content
0          False          False            False
1          False          False            False
2          False          False            False
3          False          False            False
4          False          False            False
5          False          False            False
6          False          False            False
7          False          False            False
8          False          False            False
9          False          False            False
10         False          False            False
11         False          False            False
12         False          False            False
13         False          False            False
14         False          False            False
15         False          False            False
16         False          False            False
17         False          False            False
18         False          False            False
19         F

Kita Hapus data yang missing value, maka 36 data yang kosong tersebut dihilangkan. Lalu kita load lagi data yang telah dinormalisasi.

In [6]:
#Baca data CSV dengan pandas
df = pandas.read_csv('modifiedData.csv')
Id=df['article_id']
Con= df['article_content']
Topic = df['article_topic']

Setelah itu dilakukan tahap tokenisasi yaitu memecah kalimat menjadi kata-kata yang dilakukan untuk menjadikan sebuah kalimat menjadi lebih bermakna.

In [7]:
#Pisahkan kata/tokenize data

PK= [nltk.word_tokenize(PisahKata) for PisahKata in df['article_content']]

Setelah itu masuk ketahap stopword removal/filtrasi yaitu filtrasi adalah tahap mengambil kata-kata penting

In [8]:
stopword = ['ada','adalah','adanya','adapun','agak','agaknya','agar','akan','akankah',
                 'akhir','akhiri','akhirnya','aku','akulah','amat','amatlah','anda','andalah',
                 'antar','antara','antaranya','apa','apaan','apabila','apakah','apalagi','apatah',
                 'artinya','asal','asalkan','atas','atau','ataukah','ataupun','awal','awalnya','bagai',
                 'bagaikan','bagaimana','bagaimanakah','bagaimanapun','bagi','bagian','bahkan','bahwa',
                 'bahwasanya','baik','bakal','bakalan','balik','banyak','bapak','baru','bawah','beberapa',
                 'begini','beginian','beginikah','beginilah','begitu','begitukah','begitulah','begitupun',
                 'bekerja','belakang','belakangan','belum','belumlah','benar','benarkah','benarlah','berada',
                 'berakhir','berakhirlah','berakhirnya','berapa','berapakah','berapalah','berapapun','berarti',
                 'berawal','berbagai','berdatangan','beri','berikan','berikut','berikutnya','berjumlah','berkali-kali',
                 'berkata','berkehendak','berkeinginan','berkenaan','berlainan','berlalu','berlangsung','berlebihan','bermacam',
                 'bermacam-macam','bermaksud','bermula','bersama','bersama-sama','bersiap','bersiap-siap','bertanya','bertanya-tanya','berturut','berturut-turut','bertutur','berujar','berupa','besar','betul','betulkah','biasa','biasanya','bila','bilakah','bisa','bisakah','boleh','bolehkah','bolehlah','buat','bukan','bukankah','bukanlah','bukannya','bulan','bung','cara','caranya','cukup','cukupkah','cukuplah','cuma','dahulu','dalam','dan','dapat','dari','daripada','datang','dekat','demi','demikian','demikianlah','dengan','depan','di','dia','diakhiri','diakhirinya','dialah','diantara','diantaranya','diberi','diberikan','diberikannya','dibuat','dibuatnya','didapat','didatangkan','digunakan','diibaratkan','diibaratkannya','diingat','diingatkan','diinginkan','dijawab','dijelaskan','dijelaskannya','dikarenakan','dikatakan','dikatakannya','dikerjakan','diketahui','diketahuinya','dikira','dilakukan','dilalui','dilihat','dimaksud','dimaksudkan','dimaksudkannya','dimaksudnya','diminta','dimintai','dimisalkan','dimulai','dimulailah','dimulainya','dimungkinkan','dini','dipastikan','diperbuat','diperbuatnya','dipergunakan','diperkirakan','diperlihatkan','diperlukan','diperlukannya','dipersoalkan','dipertanyakan','dipunyai','diri','dirinya','disampaikan','disebut','disebutkan','disebutkannya','disini','disinilah','ditambahkan','ditandaskan','ditanya','ditanyai','ditanyakan','ditegaskan','ditujukan','ditunjuk','ditunjuki','ditunjukkan','ditunjukkannya','ditunjuknya','dituturkan','dituturkannya','diucapkan','diucapkannya','diungkapkan','dong','dua','dulu','empat','enggak','enggaknya','entah','entahlah','guna','gunakan','hal','hampir','hanya','hanyalah','hari','harus','haruslah','harusnya','hendak','hendaklah','hendaknya','hingga','ia','ialah','ibarat','ibaratkan','ibaratnya','ibu','ikut','ingat','ingat-ingat','ingin','inginkah','inginkan','ini','inikah','inilah','itu','itukah','itulah','jadi','jadilah','jadinya','jangan','jangankan','janganlah','jauh','jawab','jawaban','jawabnya','jelas','jelaskan','jelaslah','jelasnya','jika','jikalau','juga','jumlah','jumlahnya','justru','kala','kalau','kalaulah','kalaupun','kalian','kami','kamilah','kamu','kamulah','kan','kapan','kapankah','kapanpun','karena','karenanya','kasus','kata','katakan','katakanlah','katanya','ke','keadaan','kebetulan','kecil','kedua','keduanya','keinginan','kelamaan','kelihatan','kelihatannya','kelima','keluar','kembali','kemudian','kemungkinan','kemungkinannya','kenapa','kepada','kepadanya','kesampaian','keseluruhan','keseluruhannya','keterlaluan','ketika','khususnya','kini','kinilah','kira','kira-kira','kiranya','kita','kitalah','kok','kurang','lagi','lagian','lah','lain','lainnya','lalu','lama','lamanya','lanjut','lanjutnya','lebih','lewat','lima','luar','macam','maka','makanya','makin','malah','malahan','mampu','mampukah','mana','manakala','manalagi','masa','masalah','masalahnya','masih','masihkah','masing','masing-masing','mau','maupun','melainkan','melakukan','melalui','melihat','melihatnya','memang','memastikan','memberi','memberikan','membuat','memerlukan','memihak','meminta','memintakan','memisalkan','memperbuat','mempergunakan','memperkirakan','memperlihatkan','mempersiapkan','mempersoalkan','mempertanyakan','mempunyai','memulai','memungkinkan','menaiki','menambahkan','menandaskan','menanti','menanti-nanti','menantikan','menanya','menanyai','menanyakan','mendapat','mendapatkan','mendatang','mendatangi','mendatangkan','menegaskan','mengakhiri','mengapa','mengatakan','mengatakannya','mengenai','mengerjakan','mengetahui','menggunakan','menghendaki','mengibaratkan','mengibaratkannya','mengingat','mengingatkan','menginginkan','mengira','mengucapkan','mengucapkannya','mengungkapkan','menjadi','menjawab','menjelaskan','menuju','menunjuk','menunjuki','menunjukkan','menunjuknya','menurut','menuturkan','menyampaikan','menyangkut','menyatakan','menyebutkan','menyeluruh','menyiapkan','merasa','mereka','merekalah','merupakan','meski','meskipun','meyakini','meyakinkan','minta','mirip','misal','misalkan','misalnya','mula','mulai','mulailah','mulanya','mungkin','mungkinkah','nah','naik','namun','nanti','nantinya','nyaris','nyatanya','oleh','olehnya','pada','padahal','padanya','pak','paling','panjang','pantas','para','pasti','pastilah','penting','pentingnya','per','percuma','perlu','perlukah','perlunya','pernah','persoalan','pertama','pertama-tama','pertanyaan','pertanyakan','pihak','pihaknya','pukul','pula','pun','punya','rasa','rasanya','rata','rupanya','saat','saatnya','saja','sajalah','saling','sama','sama-sama','sambil','sampai','sampai-sampai','sampaikan','sana','sangat','sangatlah','satu','saya','sayalah','se','sebab','sebabnya','sebagai','sebagaimana','sebagainya','sebagian','sebaik','sebaik-baiknya','sebaiknya','sebaliknya','sebanyak','sebegini','sebegitu','sebelum','sebelumnya','sebenarnya','seberapa','sebesar','sebetulnya','sebisanya','sebuah','sebut','sebutlah','sebutnya','secara','secukupnya','sedang','sedangkan','sedemikian','sedikit','sedikitnya','seenaknya','segala','segalanya','segera','seharusnya','sehingga','seingat','sejak','sejauh','sejenak','sejumlah','sekadar','sekadarnya','sekali','sekali-kali','sekalian','sekaligus','sekalipun','sekarang','sekarang','sekecil','seketika','sekiranya','sekitar','sekitarnya','sekurang-kurangnya','sekurangnya','sela','selain','selaku','selalu','selama','selama-lamanya','selamanya','selanjutnya','seluruh','seluruhnya','semacam','semakin','semampu','semampunya','semasa','semasih','semata','semata-mata','semaunya','sementara','semisal','semisalnya','sempat','semua','semuanya','semula','sendiri','sendirian','sendirinya','seolah','seolah-olah','seorang','sepanjang','sepantasnya','sepantasnyalah','seperlunya','seperti','sepertinya','sepihak','sering','seringnya','serta','serupa','sesaat','sesama','sesampai','sesegera','sesekali','seseorang','sesuatu','sesuatunya','sesudah','sesudahnya','setelah','setempat','setengah','seterusnya','setiap','setiba','setibanya','setidak-tidaknya','setidaknya','setinggi','seusai','sewaktu','siap','siapa','siapakah','siapapun','sini','sinilah','soal','soalnya','suatu','sudah','sudahkah','sudahlah','supaya','tadi','tadinya','tahu','tahun','tak','tambah','tambahnya','tampak','tampaknya','tandas','tandasnya','tanpa','tanya','tanyakan','tanyanya','tapi','tegas','tegasnya','telah','tempat','tengah','tentang','tentu','tentulah','tentunya','tepat','terakhir','terasa','terbanyak','terdahulu','terdapat','terdiri','terhadap','terhadapnya','teringat','teringat-ingat','terjadi','terjadilah','terjadinya','terkira','terlalu','terlebih','terlihat','termasuk','ternyata','tersampaikan','tersebut','tersebutlah','tertentu','tertuju','terus','terutama','tetap','tetapi','tiap','tiba','tiba-tiba','tidak','tidakkah','tidaklah','tiga','tinggi','toh','tunjuk','turut','tutur','tuturnya','ucap','ucapnya','ujar','ujarnya','umum','umumnya','ungkap','ungkapnya','untuk','usah','usai','waduh','wah','wahai','waktu','waktunya','walau','walaupun','wong','yaitu','yakin','yakni','yang']

sw=[]

for conten in PK:
    kata= list(filter(lambda x: x not in stopword, conten))
    sw.append(kata)

df['stopword'] = sw

Setelah menghilangkan kata-kata yang tidak penting akan tersisa kata-kata yang penting saja pada data. lalu kita ubah data dari list ke string untuk proses selanjutnya.

In [9]:
#Ubah list ke str
string = map(' '.join, sw)

## Feature Extraction

Lalu setelah itu masuk ketahap feature extraction dimana kata-kata pada data diubah menjadi vector model menggunakan term frequency(tf) dan term frequency-inverse document frequency(tf-idf). tf(countvector/raw count) cocok digunakan untuk algoritma LDA karena LDA menggunakan model probabilitas sehingga hanya membutuhkan raw count saja, dan algoritma NMF menggunakan tfidf dalam pemrosesanya karena berdasarkan linear algebra dalam pemodelanya.

In [10]:
#CountVector/raw count
count_vect = CountVectorizer()
count = count_vect.fit_transform(string)
tf_feature_names = count_vect.get_feature_names()

#tf-idf
string = map(' '.join, sw)

tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(string)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

## Modelling NMF dan LDA

Dilakukan modelling dengan menggunakan algoritma Non-negative Matrix Factorization (NMF) dan juga Latent Dirichlet Allocation (LDA).

Kita coba dengan NMF dan LDA, pada NMF merepresentasikan Dokumen menjadi beberapa topik berdasarkan linear algebra. 
Sedangkan pada LDA merepresentasikan Dokumen menjadi beberapa topik dengan model probabilitas. Pada task ini jumlah topik sebesar 29 topik.

In [11]:
no_topics = 29

# Coba algoritma NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Coba algoritma LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(count)

c:\users\user\appdata\local\programs\python\python35\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [12]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10

Menampilkan hasil dari NMF.

In [13]:
display_topics(nmf, tfidf_feature_names, no_top_words)

Topic 0:
jemaah, haji, saudi, madinah, menag, arab, petugas, makkah, bus, indonesia
Topic 1:
united, liverpool, musim, pemain, league, premier, city, arsenal, liga, manchester
Topic 2:
indonesia, negara, perusahaan, pemerintah, pt, masyarakat, 2017, jakarta, presiden, menteri
Topic 3:
saham, indeks, poin, ihsg, rp, turun, perdagangan, persen, menguat, level
Topic 4:
anda, tubuh, penyakit, kanker, kulit, kesehatan, makanan, darah, buah, obat
Topic 5:
gamat, jelly, qnc, pemesanan, penyakit, obat, teripang, no, botol, herbal
Topic 6:
wafat, jantung, madinah, agustus, serangan, bin, 2017, rs, usia, sub
Topic 7:
jamaah, haji, saudi, ibadah, arab, madinah, kesehatan, makkah, calon, mekkah
Topic 8:
timnas, pemain, 19, egy, indra, 22, indonesia, thailand, piala, tim
Topic 9:
haji, dana, infrastruktur, investasi, bpkh, syariah, pengelolaan, diinvestasikan, keuangan, pemerintah
Topic 10:
rohingya, myanmar, rakhine, bangladesh, kyi, suu, desa, kekerasan, pengungsi, etnis
Topic 11:
pebalap, balapa

Dapat dilihat hasil 30 topik dari NMF.

Menampilkan hasil dari LDA.

In [14]:
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
mobil, listrik, motor, kendaraan, unit, mesin, toyota, mitsubishi, model, xpander
Topic 1:
pohon, kucing, mbah, probolinggo, manusia, sunan, harta, pemimpin, daffa, torem
Topic 2:
the, and, you, vicky, tulis, ariel, we, acha, be, that
Topic 3:
indonesia, negara, orang, jakarta, pemerintah, masyarakat, salah, 2017, 10, memiliki
Topic 4:
penyakit, anda, obat, gamat, jelly, tubuh, qnc, memiliki, kanker, kesehatan
Topic 5:
terminal, pelabuhan, karyawan, tanjung, mogok, jict, bongkar, priok, aro, rhb
Topic 6:
anak, orang, film, ya, lagu, namun, sang, foto, salah, berusia
Topic 7:
perempuan, weinstein, laki, gunungsitoli, mahrez, yordania, beltim, steinhaus, doel, jedi
Topic 8:
haji, jemaah, 2017, wafat, usia, makkah, jamaah, gangguan, jantung, agustus
Topic 9:
ponsel, pengguna, aplikasi, kamera, teknologi, iphone, layanan, fitur, google, indonesia
Topic 10:
pemain, gol, laga, musim, tim, bola, pertandingan, berhasil, menit, liga
Topic 11:
rp, persen, as, saham, triliun, dolar, harg

Dapat dilihat hasil 30 topik dari LDA.

## Evaluasi
Program diatas merupakan unsupervised learning sehingga susah untuk melakukan evaluasi. Oleh karena itu, evaluasi dilakukan dengan membandingkan seberapa relevan hasil dari LDA dan NMF terhadap topic yang sebenernya.

Coba lihat hasil dari algoritma NMF.

In [15]:
tfidf_NMF=nmf.transform(tfidf)

dok_topic = [x.argmax() for x in tfidf_NMF]

topic_dict = {}
for topic_idx, topic in enumerate(lda.components_[:no_topics]):
    topic_dict[topic_idx] = ", ".join([tfidf_feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])
    
df["nmf_topic"] = [topic_dict[i] for i in dok_topic]

df.sample(10, random_state=10)

,article_id,article_topic,article_content,stopword,nmf_topic
3358,560368,Lifestyle,"Banyak yang bilang, jodoh tak bisa ditebak. A...","[Banyak, bilang, ,, jodoh, ditebak, ., Ada, pa...","shankly, makoto, busby, guelenist, cassini, ep..."
9330,93185907,Teknologi,\r\nDemonstrasi yang berlangsung di Charlotte...,"[Demonstrasi, Charlottesville, ,, Virginia, ,,...","ponsel, pengguna, aplikasi, kamera, teknologi,..."
6641,93207792,Personal,"Bagi pemuda Indonesia yang cinta tanah air, ...","[Bagi, pemuda, Indonesia, cinta, tanah, air, ,...","the, and, you, vicky, tulis, ariel, we, acha, ..."
1629,93219927,Hiburan,Keluarga musisi Anji tengah berbahagia. Putra...,"[Keluarga, musisi, Anji, berbahagia, ., Putra,...","shankly, makoto, busby, guelenist, cassini, ep..."
6460,93211703,Ekonomi,Rapat Umum Pemegang Saham Luar Biasa (RUPSLB)...,"[Rapat, Umum, Pemegang, Saham, Luar, Biasa, (,...","the, and, you, vicky, tulis, ariel, we, acha, ..."
2926,93181624,Ekonomi,Hingga saat ini pihak PT Jakarta Internationa...,"[Hingga, PT, Jakarta, International, Container...","the, and, you, vicky, tulis, ariel, we, acha, ..."
1377,93202091,Teknologi,Apple sepertinya akan memperkenalkan sebuah p...,"[Apple, memperkenalkan, ponsel, menandai, 10, ...","serangan, tewas, al, isis, korban, bom, kelomp..."
6111,93213899,Internasional,Siti Aisyah akan kembali menjalani pengadilan...,"[Siti, Aisyah, menjalani, pengadilan, Senin, p...","dion, alana, fiona, tupperware, taliban, nato,..."
1487,93230324,Ekonomi,Dunia bisnis Malaysia dihebohkan oleh pengund...,"[Dunia, bisnis, Malaysia, dihebohkan, pengundu...","the, and, you, vicky, tulis, ariel, we, acha, ..."
1756,93186986,Haji,Menag Lukman menerima cinderamta dari Sekjen ...,"[Menag, Lukman, menerima, cinderamta, Sekjen, ...","the, and, you, vicky, tulis, ariel, we, acha, ..."


Hasil dari algoritma NMF dapat dilihat dari 10 sampel diatas, dimana ada yang masih relevan dengan topik dan ada yang tidak.

Coba dengan algoritma LDA.

In [16]:
count_vect_lda=lda.transform(count)

dok_topic = [x.argmax() for x in count_vect_lda]

topic_dict = {}
for topic_idx, topic in enumerate(lda.components_[:no_topics]):
    topic_dict[topic_idx] = ", ".join([tf_feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])
    
df["lda_topic"] = [topic_dict[i] for i in dok_topic]

df.sample(10, random_state=10)

,article_id,article_topic,article_content,stopword,nmf_topic,lda_topic
3358,560368,Lifestyle,"Banyak yang bilang, jodoh tak bisa ditebak. A...","[Banyak, bilang, ,, jodoh, ditebak, ., Ada, pa...","shankly, makoto, busby, guelenist, cassini, ep...","anak, orang, film, ya, lagu, namun, sang, foto..."
9330,93185907,Teknologi,\r\nDemonstrasi yang berlangsung di Charlotte...,"[Demonstrasi, Charlottesville, ,, Virginia, ,,...","ponsel, pengguna, aplikasi, kamera, teknologi,...","indonesia, negara, orang, jakarta, pemerintah,..."
6641,93207792,Personal,"Bagi pemuda Indonesia yang cinta tanah air, ...","[Bagi, pemuda, Indonesia, cinta, tanah, air, ,...","the, and, you, vicky, tulis, ariel, we, acha, ...","indonesia, negara, orang, jakarta, pemerintah,..."
1629,93219927,Hiburan,Keluarga musisi Anji tengah berbahagia. Putra...,"[Keluarga, musisi, Anji, berbahagia, ., Putra,...","shankly, makoto, busby, guelenist, cassini, ep...","anak, orang, film, ya, lagu, namun, sang, foto..."
6460,93211703,Ekonomi,Rapat Umum Pemegang Saham Luar Biasa (RUPSLB)...,"[Rapat, Umum, Pemegang, Saham, Luar, Biasa, (,...","the, and, you, vicky, tulis, ariel, we, acha, ...","indonesia, negara, orang, jakarta, pemerintah,..."
2926,93181624,Ekonomi,Hingga saat ini pihak PT Jakarta Internationa...,"[Hingga, PT, Jakarta, International, Container...","the, and, you, vicky, tulis, ariel, we, acha, ...","indonesia, negara, orang, jakarta, pemerintah,..."
1377,93202091,Teknologi,Apple sepertinya akan memperkenalkan sebuah p...,"[Apple, memperkenalkan, ponsel, menandai, 10, ...","serangan, tewas, al, isis, korban, bom, kelomp...","ponsel, pengguna, aplikasi, kamera, teknologi,..."
6111,93213899,Internasional,Siti Aisyah akan kembali menjalani pengadilan...,"[Siti, Aisyah, menjalani, pengadilan, Senin, p...","dion, alana, fiona, tupperware, taliban, nato,...","indonesia, negara, orang, jakarta, pemerintah,..."
1487,93230324,Ekonomi,Dunia bisnis Malaysia dihebohkan oleh pengund...,"[Dunia, bisnis, Malaysia, dihebohkan, pengundu...","the, and, you, vicky, tulis, ariel, we, acha, ...","indonesia, negara, orang, jakarta, pemerintah,..."
1756,93186986,Haji,Menag Lukman menerima cinderamta dari Sekjen ...,"[Menag, Lukman, menerima, cinderamta, Sekjen, ...","the, and, you, vicky, tulis, ariel, we, acha, ...","indonesia, negara, orang, jakarta, pemerintah,..."


Hampir sama dengan NMF dimana terdapat relevansi antara hasil dan topic sebenarnya, namun ada juga yang tidak.

## Future Works
- Mencoba metode clustering, apakah bisa lebih bagus dengan metode tersebut.
- Menggunakan fungsi regex untuk dapat mengklasifikasikan berdasarkan kata-kata kunci yang termasuk dalam topik yang mana
- Menggunakan algoritma machine learning lain dan mecoba menerapkan deep learning dalam prediksi
- Mencoba menambah proses yang diperlukan pada preprocessing data seperti stemming, dan lain sebagainya
- Mencoba Tuning parameter pada countvectorizer, tf-idf dan algoritma machine learning